In [64]:
(ql:quickload :kenzo)

To load "kenzo":
  Load 1 ASDF system:
    kenzo
; Loading "kenzo"



(:KENZO)

In [64]:
(cat:cat-init)


---done---

NIL

### 2.2.1 Representation of a reduction

In [64]:
(documentation 'cat:reduction 'type)

"----------------------------------------------------------------[class-doc]
REDUCTION
Slots: (tcc bcc f g h idnm orgn)
Instances of this class represent reductions of chain complexes.

     ^    h     s ^                 f ° g = 1
     C  ----->    C                          C
      ^                     h ° d + d ° h = 1  - g ° f
    | |                                      ^
  f | | g                                    C
    | |                             f ° h = 0
    v                               h ° g = 0
     C                              h ° h = 0

The class has 7 slots:
                                                               ^
1. TCC, an object of type CHAIN-COMPLEX, the top chain complex C. Its reader
   function is TCC1.

2. BCC, an object of type CHAIN-COMPLEX, the bottom chain complex C. Its reader
   function is BCC1.

3. F, an object of type MORPHISM representing the morphism f. Its reader
   function is F1.

4. G, an object of type MORPHISM representing the m

### 2.2.2 The function $\texttt{build-rdct}$

In [64]:
(documentation 'cat:build-rdct 'function)

"--------------------------------------------------------------[function-doc]
BUILD-RDCT
Args: (&key f g h orgn)
Returns an instance of the class REDUCTION. The keyword arguments are as
follows:

:F F, an object of type morphism representing the morphism f of a reduction

:G G, an object of type morphism representing the morphism g of a reduction

:H H, an object of type morphism representing the morphism h of a reduction

:ORGN ORGN, a list containing a relevant and carefully chosen comment about
            the origin of the chain complex. This comment should be unique
            for a Kenzo session (between calls of CAT-INIT), as it is used
            for caching purposes.

Use this function instead of creating instances via the standard constructor
MAKE-INSTANCE.
------------------------------------------------------------------------------"

### 2.2.4 Verification functions

In [64]:
(documentation 'cat:pre-check-rdct 'function)

"--------------------------------------------------------------[function-doc]
PRE-CHECK-RDCT
Args: (rdct)
Assigns the following Lisp global variables, using morphisms from RDCT and the
differentials and identity morphisms of the underlying chain complexes,
according to these formulas:

*TDD*         = d  ° d
                 ^    ^
                 C    C

*BDD*         = d  ° d
                 C    C

*ID-FG*       = Id  - f ° g
                  C

*ID-GF-DH-HD* = Id  - g ° f - (d ° h + h ° d )
                  ^             ^           ^
                  C             C           C

*HH*          = h ° h
*FH*          = f ° h
*HG*          = h ° g

*DF-FD*       = d  ° f - f ° d
                 C            ^
                              C

*DG-GD*       = d  ° g - g ° d
                 ^            C
                 C

------------------------------------------------------------------------------"

In [64]:
(documentation 'cat:check-rdct-no-wait 'function)

"--------------------------------------------------------------[function-doc]
CHECK-RDCT-NO-WAIT
Args: ()
Maps combinations of the top and bottom chain complexes of a reduction using
the morphisms created by PRE-CHECK-RDCT. Having no parameters, the
combinations must be provided via the Lisp global variables *TC* and *BC*,
for the top and bottom chain complex, respectively. If the morphisms are
coherent, the result of each mapping is a null combination.
------------------------------------------------------------------------------"

#### Example

In [64]:
(defun cdelta (dmns)
    (cat:build-chcm
     :cmpr #'cat:l-cmpr
     :basis :locally-effective
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (cat:make-cmbn
             :degr (1- degr)
             :list (do ((rslt cat:+empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(locally effective version of C_* delta ,dmns)))

CDELTA

In [64]:
(defun make-f (tdmns bdmns)
    (cat:build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta bdmns) :degr 0
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (< pos degr)
                             (cat:zero-cmbn degr)
                             (cat:cmbn degr 1 (nconc (butlast gmsm) (list bdmns))))
                         (cat:cmbn degr 1 gmsm))))
     :strt :gnrt
     :orgn `(projection delta ,tdmns => delta ,bdmns)))

MAKE-F

In [64]:
(defun make-g (tdmns bdmns)
    (cat:build-mrph
     :sorc (cdelta bdmns) :trgt (cdelta tdmns) :degr 0
     :intr #'identity
     :strt :cmbn
     :orgn `(injection delta ,bdmns => delta ,tdmns)))

MAKE-G

In [64]:
(defun make-h (tdmns bdmns)
    (cat:build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta tdmns) :degr +1
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (member bdmns gmsm)
                             (cat:zero-cmbn (1+ degr))
                             (cat:cmbn (1+ degr) (cat:-1-expt-n pos)
                                       (append (subseq gmsm 0 pos) (list bdmns)
                                               (subseq gmsm pos))))
                         (cat:zero-cmbn (1+ degr)))))
     :strt :gnrt
     :orgn `(homotopy for delta ,tdmns => ,bdmns)))

MAKE-H

In [64]:
(defun make-rdct (tdmns bdmns)
    (let ((rdct (cat:build-rdct
                 :f (make-f tdmns bdmns)
                 :g (make-g tdmns bdmns)
                 :h (make-h tdmns bdmns)
                 :orgn `(reduction delta ,tdmns ,bdmns))))
         rdct))

MAKE-RDCT

In [64]:
(cat:def rdct (make-rdct 6 3))

[K8 Reduction K1 => K3]

In [64]:
(cat:orgn rdct)

(REDUCTION DELTA 6 3)

In [64]:
(cat:pre-check-rdct rdct)


---done---

NIL

In [64]:
(setf cat:*tc* (cat:cmbn 2 1 '(0 1 2) 10 '(1 2 3) 100 '(1 2 4) 1000 '(2 3 4)))


----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------


In [64]:
(setf cat:*bc* (cat:cmbn 3 4 '(0 1 2 3)))


----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------


In [64]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
R

NIL

In [64]:
(cat:def trdct (make-rdct 6 4))

[K35 Reduction K1 => K30]

In [64]:
(cat:def brdct (make-rdct 4 3))

[K39 Reduction K30 => K3]

In [64]:
(setf rdct (cat:cmps brdct trdct))

[K45 Reduction K1 => K3]

In [64]:
(cat:pre-check-rdct rdct)


---done---

NIL

In [64]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
R

NIL

### 2.4.1 Useful functions related to the perturbation lemma
#### Example

In [64]:
(cat:def hmeq (cat:trivial-hmeq (cdelta 4)))

[K66 Homotopy-Equivalence K30 <= K30 => K30]

In [64]:
(setf hmeq (cat:add hmeq (cat:opps (cat:dffr (cdelta 4)))))

[K74 Homotopy-Equivalence K69 <= K69 => K69]

In [64]:
(cat:gnrt-? (cat:dffr (cat:rbcc hmeq)) 3 '(0 1 2 3))


----------------------------------------------------------------------{CMBN 2}
------------------------------------------------------------------------------


### 2.5.1 Representation of a combination in a bicone

In [64]:
(cat:def comb-bic (cat:cmbn 3 2 (cat:bcnb 'b1) 4 (cat:bcnb 'b1) 6 (cat:bcnb 'b3)
                              3 (cat:bcnb 'c1) 5 (cat:bcnb 'c1) 7 (cat:bcnb 'd1)))


----------------------------------------------------------------------{CMBN 3}
<2 * <BcnB B1>>
<4 * <BcnB B1>>
<6 * <BcnB B3>>
<3 * <BcnB C1>>
<5 * <BcnB C1>>
<7 * <BcnB D1>>
------------------------------------------------------------------------------


In [64]:
(cat:cmbn-list comb-bic)

((2 . <BcnB B1>) (4 . <BcnB B1>) (6 . <BcnB B3>) (3 . <BcnB C1>) (5 . <BcnB C1>) (7 . <BcnB D1>))

### 2.5.2 Useful functions and macros for bicones
#### Examples

In [64]:
(cat:def comb-b (cat:cmbn 3 2 'b1 4 'b2 6 'b3))


----------------------------------------------------------------------{CMBN 3}
<2 * B1>
<4 * B2>
<6 * B3>
------------------------------------------------------------------------------


In [64]:
(cat:def comb-c (cat:cmbn 4 3 'c1 5 'c2))


----------------------------------------------------------------------{CMBN 4}
<3 * C1>
<5 * C2>
------------------------------------------------------------------------------


In [64]:
(cat:def comb-d (cat:cmbn 3 7 'd1))


----------------------------------------------------------------------{CMBN 3}
<7 * D1>
------------------------------------------------------------------------------


In [64]:
(cat:def comb-bic (cat:make-bicn-cmbn comb-b comb-c comb-d))


----------------------------------------------------------------------{CMBN 3}
<2 * <BcnB B1>>
<4 * <BcnB B2>>
<6 * <BcnB B3>>
<3 * <BcnC C1>>
<5 * <BcnC C2>>
<7 * <BcnD D1>>
------------------------------------------------------------------------------


In [64]:
(cat:bicn-cmbn-cmbnb comb-bic)


----------------------------------------------------------------------{CMBN 3}
<2 * B1>
<4 * B2>
<6 * B3>
------------------------------------------------------------------------------


In [64]:
(cat:bicn-cmbn-cmbnc comb-bic)


----------------------------------------------------------------------{CMBN 4}
<3 * C1>
<5 * C2>
------------------------------------------------------------------------------


In [64]:
(cat:bicn-cmbn-cmbnd comb-bic)


----------------------------------------------------------------------{CMBN 3}
<7 * D1>
------------------------------------------------------------------------------


In [64]:
(multiple-value-bind (b c d)
    (cat:dispatch-bicn-cmbn comb-bic)
    (list b c d))

(
----------------------------------------------------------------------{CMBN 3}
<2 * B1>
<4 * B2>
<6 * B3>
------------------------------------------------------------------------------
 
----------------------------------------------------------------------{CMBN 4}
<3 * C1>
<5 * C2>
------------------------------------------------------------------------------
 
----------------------------------------------------------------------{CMBN 3}
<7 * D1>
------------------------------------------------------------------------------
)

## 2.6 Construction of abicone from 2 reductions
#### Example

In [64]:
(cat:cat-init)


---done---

NIL

In [64]:
(defun cdelta (dmns)
    (cat:build-chcm
     :cmpr #'cat:l-cmpr
     :basis #'(lambda (n)
                      (mapcar #'cat:dlop-int-ext (funcall (cat:delta-n-basis dmns) n)))
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (cat:make-cmbn
             :degr (1- degr)
             :list (do ((rslt cat:+empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(Effective version of C_* delta ,dmns)))

CDELTA

In [64]:
(cat:def delta3 (cdelta 3))

[K1 Chain-Complex]

In [64]:
(cat:basis delta3 0)

((0) (1) (2) (3))

In [64]:
(cat:basis delta3 1)

((0 1) (0 2) (1 2) (0 3) (1 3) (2 3))

In [64]:
(cat:basis delta3 2)

((0 1 2) (0 1 3) (0 2 3) (1 2 3))

In [64]:
(cat:basis delta3 3)

((0 1 2 3))

In [64]:
(cat:basis delta3 4)

NIL

In [64]:
(cat:def bic (cat:bicone (make-rdct 3 2) (make-rdct 4 2)))

[K15 Chain-Complex]

In [64]:
(cat:basis bic 0)

(<BcnB (0)> <BcnB (1)> <BcnB (2)> <BcnB (3)> <BcnC (0 1)> <BcnC (0 2)> <BcnC (1 2)> <BcnD (0)> <BcnD (1)> <BcnD (2)> ...)

In [64]:
(cat:basis bic 1)

(<BcnB (0 1)> <BcnB (0 2)> <BcnB (1 2)> <BcnB (0 3)> <BcnB (1 3)> <BcnB (2 3)> <BcnC (0 1 2)> <BcnD (0 1)> <BcnD (0 2)> <BcnD (1 2)> ...)

In [64]:
(cat:basis bic 4)

(<BcnD (0 1 2 3 4)>)

In [64]:
(cat:? bic (cat:cmbn 2 3 (cat:bcnb '(0 1 3)) 4 (cat:bcnc '(0 1 2 3)) 5 (cat:bcnd '(0 1 4))))


----------------------------------------------------------------------{CMBN 1}
<3 * <BcnB (0 1)>>
<-3 * <BcnB (0 3)>>
<3 * <BcnB (1 3)>>
<12 * <BcnC (0 1 2)>>
<-4 * <BcnC (0 1 3)>>
<4 * <BcnC (0 2 3)>>
<-4 * <BcnC (1 2 3)>>
<5 * <BcnD (0 1)>>
<-5 * <BcnD (0 4)>>
<5 * <BcnD (1 4)>>
------------------------------------------------------------------------------


In [64]:
(cat:? bic (cat:? bic (cat:cmbn 2 3 (cat:bcnb '(0 1 3)) 4 (cat:bcnc '(0 1 2 3)) 5 (cat:bcnd '(0 1 4)))))


----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------


## 2.7 Composition of homotopy equivalences
#### Example

In [64]:
(cat:cat-init)


---done---

NIL

In [64]:
(cat:def c (cat:build-chcm
            :cmpr #'cat:s-cmpr
            :basis #'(lambda (dmns) (declare (ignore dmns)) '(a))
            :bsgn 'a
            :intr-dffr #'cat:zero-intr-dffr
            :strt :cmbn
            :orgn '(c)))

[K1 Chain-Complex]

In [64]:
(cat:def h1 (cat:trivial-hmeq c))

[K6 Homotopy-Equivalence K1 <= K1 => K1]

In [64]:
(cat:def h2 (cat:cmps h1 h1))

[K17 Homotopy-Equivalence K1 <= K7 => K1]

In [64]:
(cat:pre-check-rdct (cat:lrdct h2))


---done---

NIL

In [64]:
(setf cat:*tc* (cat:cmbn 3 1 (cat:bcnb 'a) 10 (cat:bcnc 'a) 100 (cat:bcnd 'a)))


----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB A>>
<10 * <BcnC A>>
<100 * <BcnD A>>
------------------------------------------------------------------------------


In [64]:
(setf cat:*bc* (cat:cmbn 3 1 'a))


----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------


In [64]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB A>>
<10 * <BcnC A>>
<100 * <BcnD A>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
Result: 
--------------

NIL

In [64]:
(cat:pre-check-rdct (cat:rrdct h2))


---done---

NIL

In [64]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB A>>
<10 * <BcnC A>>
<100 * <BcnD A>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
Result: 
--------------

NIL

In [64]:
(cat:def h3 (cat:cmps h2 h2))

[K65 Homotopy-Equivalence K1 <= K55 => K1]

In [64]:
(setf cat:*tc* (cat:cmbn 3 1 (cat:bcnb (cat:bcnb 'a)) 10 (cat:bcnb (cat:bcnc 'a)) 100 (cat:bcnb (cat:bcnd 'a))
                         1000 (cat:bcnc 'a)
                         10000 (cat:bcnd (cat:bcnb 'a)) 5234 (cat:bcnd (cat:bcnc 'a))
                         223 (cat:bcnd (cat:bcnd 'a))))


----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB <BcnB A>>>
<10 * <BcnB <BcnC A>>>
<100 * <BcnB <BcnD A>>>
<1000 * <BcnC A>>
<10000 * <BcnD <BcnB A>>>
<5234 * <BcnD <BcnC A>>>
<223 * <BcnD <BcnD A>>>
------------------------------------------------------------------------------


In [64]:
(cat:pre-check-rdct (cat:lrdct h3))


---done---

NIL

In [64]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB <BcnB A>>>
<10 * <BcnB <BcnC A>>>
<100 * <BcnB <BcnD A>>>
<1000 * <BcnC A>>
<10000 * <BcnD <BcnB A>>>
<5234 * <BcnD <BcnC A>>>
<223 * <BcnD <BcnD A>>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
----------

NIL

In [64]:
(cat:pre-check-rdct (cat:rrdct h3))


---done---

NIL

In [64]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB <BcnB A>>>
<10 * <BcnB <BcnC A>>>
<100 * <BcnB <BcnD A>>>
<1000 * <BcnC A>>
<10000 * <BcnD <BcnB A>>>
<5234 * <BcnD <BcnC A>>>
<223 * <BcnD <BcnD A>>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
----------

NIL